<a href="https://colab.research.google.com/github/SonjaWangJQ/Keyboard-ML/blob/main/4_cepstrum_on_classifers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LOAD DATA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:

from sklearn.preprocessing import StandardScaler

from sklearn.utils import shuffle

import math, random
import torch
import torchaudio
import librosa
from torchaudio import transforms
from IPython.display import Audio
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, random_split
import csv
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import dct

In [ ]:


class AudioUtil():
  # ----------------------------
  # Load an audio file. Return the signal as a tensor and the sample rate
  # ----------------------------

  @staticmethod
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    return (sig, sr)





  @staticmethod
  def time_shift(aud, shift_limit):
    sig,sr = aud
    _, sig_len = sig.shape
    shift_amt = int(random.random() * shift_limit * sig_len)
    return (sig.roll(shift_amt), sr)



  @staticmethod
  def compute_mfcc(aud, n_mfcc=32, window_size=0.01, step_size=0.0025, n_filters=32):

            sig, sr = aud
            n_fft = int(sr * window_size)
            hop_length = int(sr * step_size)

            mfccs = librosa.feature.mfcc(
                y=sig.numpy(),
                sr=sr,
                n_mfcc=n_mfcc,
                n_fft=n_fft,
                hop_length=hop_length,
                n_mels=n_filters
            )
            return mfccs.T


  @staticmethod
  def compute_stft(aud, n_fft=2048, hop_length=512, window='hann'):
    sig, sr = aud
    audio = sig.squeeze().numpy()

    # Compute STFT
    stft_result = librosa.core.stft(audio, n_fft=2048, hop_length=512, window='hann')

    # Get magnitude spectrum
    magnitude = np.abs(stft_result)

    # Get frequency axis
    freqs = librosa.core.fft_frequencies(sr=sr, n_fft=2048)

    return freqs, magnitude

  @staticmethod
  def compute_cepstrum(aud,window_size=0.01,step_size=0.0025):
        sig, sr = aud
        n_fft = int(sr * window_size)
        hop_length = int(sr * step_size)

        # 计算STFT（功率谱）
        stft_result = librosa.core.stft(sig.numpy(), n_fft=n_fft, hop_length=hop_length, window='hann')

        # 计算对数功率谱
        log_power_spectrum = np.log(np.abs(stft_result)**2)

        # 计算Cepstrum（倒谱）
        cepstrum = librosa.core.istft(log_power_spectrum, hop_length=hop_length, window='hann')

        return cepstrum.T




In [ ]:

# ----------------------------
# SoundDS +10ms
# ----------------------------
class SoundDS(Dataset):

  def __init__(self, csv_filename):
    self.csv_filename = csv_filename
    # self.train_indices = train_indices  # 训练集的索引列表
    self.sr = 44100
    self.channel = 1
    # self.shift_pct = 0.01   # +10ms
    self.audio_file, self.class_id = self.load_datapath()


  def load_datapath(self):
        audio_file, class_id = [], []
        with open(self.csv_filename) as f:
            reader = csv.reader(f)
            next(reader)  # Skip the header line
            for row in reader:
                i, l = row
                audio_file.append(i)
                class_id.append(int(l))  # Convert class ID to integer
        return audio_file, class_id

  def __len__(self):
    return len(self.audio_file)


  def get_class_counts(self):
        class_counts = {}
        for cls_id in self.class_id:
            if cls_id in class_counts:
                class_counts[cls_id] += 1
            else:
                class_counts[cls_id] = 1
        return class_counts





  def __getitem__(self, idx):

    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path，Get the Class ID

    # Get the Class ID
    audio_path = self.audio_file[idx]
    class_id = self.class_id[idx]


    aud = AudioUtil.open(audio_path)
    # shift_aud = AudioUtil.time_shift(aud, self.shift_pct)
    # mfcc_aud = AudioUtil.compute_mfcc(shift_aud)
    # mfcc_data = np.array(mfcc_aud)
    # fft_audio = AudioUtil.compute_fft(shift_aud )
    # fft_data = np.array(fft_audio)
    cepstrum_aud = AudioUtil.compute_cepstrum(aud)

    return cepstrum_aud, class_id




---



In [ ]:
# shift  append into X,y
csv_filename = "/content/drive/MyDrive/collect_machinekeyboard/collect/piece2_441.csv"
sound_dataset_shift= SoundDS(csv_filename)
X = []
y = []
# path = []


for idx in range(len(sound_dataset_shift)):
    mfcc_data, class_id = sound_dataset_shift[idx]
    X.append(mfcc_data)
    y.append(class_id)
    # path.append(audio_path)



In [ ]:
X = np.array(X)
y = np.array(y)
# path = np.array(path)
# check length
assert X.shape[0] == len(y), "Shape mismatch between X and y"
# assert X.shape[0] == len(path), "Shape mismatch between X and path"
print(X.shape)

X_reshaped = X.reshape(X.shape[0], -1)  # 将 X 重新塑造为二维矩阵

# 拆分数据集
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42, stratify=y)
print(X_train.shape)


(1467, 4400, 1)
(1173, 4400)


这时我们得到了train data路径, path-train(0.2*1407,)



---







---






In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # 标准化
X_test_scaled = scaler.transform(X_test) # keep it
# X_val_scaled = scaler.transform(X_val)
print(X_train_scaled.shape, X_test_scaled.shape)

(1173, 4400) (294, 4400)


In [ ]:
print(y_train.shape,y_test.shape)

(1173,) (294,)


## 使用原数据训练结果 MFCC

### svm

In [ ]:
#SVM model 使用原数据训练结果
#
# Best parameters: {'C': 10, 'kernel': 'rbf'} 0.84
#Mfcc: aud, n_mfcc=32, window_size=0.01, step_size=0.0025, n_filters=64
#  parameter grid for grid search
from sklearn.metrics import top_k_accuracy_score


param_grid = {
    # 'C': [1, 5, 10, 100],
    'C': [1],
    'kernel': ['linear'],
    'gamma': ['scale']
    # 'gamma':[0.001]
    }

#  SVM model
svm = SVC()

#  grid search object with 10-fold cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=10, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

#  best model from grid search
best_model = grid_search.best_estimator_

# Predict using the best model
y_pred = best_model.predict(X_test_scaled)
target_names = [f'Class {i}' for i in range(36)]
#   classification report
cr = classification_report(y_test, y_pred, target_names=target_names)
# N = 5  # Change this to the desired value
# top_n_accuracy = top_k_accuracy_score(y_test, best_model.decision_function(X_test_scaled), k=N)



N_values = [1, 2, 3, 4, 5]
top_n_accuracies = [top_k_accuracy_score(y_test, best_model.decision_function(X_test_scaled), k=N) for N in N_values]

# Print results
print("Best parameters:", grid_search.best_params_)
print("Classification Report:")
print(cr)
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Best parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.67      0.25      0.36         8
     Class 1       0.50      0.56      0.53         9
     Class 2       0.22      0.44      0.30         9
     Class 3       0.24      0.40      0.30        10
     Class 4       0.60      0.33      0.43         9
     Class 5       1.00      0.12      0.22         8
     Class 6       0.15      0.22      0.18         9
     Class 7       0.30      0.70      0.42        10
     Class 8       0.40      0.80      0.53        10
     Class 9       0.25      0.20      0.22        10
    Class 10       1.00      0.29      0.44         7
    Class 11       0.21      0.33      0.26         9
    Class 12       0.00      0.00      0.00         8
    Class 13       1.00      0.29      0.44         7
    Class 14       0.75      0.33      0.46         9
    Class 15       0.70      0.78      0.74     

### Multinomial Logistic Regression

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_digits
from sklearn.metrics import classification_report
from sklearn.utils import shuffle

In [ ]:
#数据 origial

# Multinomial Logistic Regression  X,y 0.2 ；L2；10 fold across MFCC

#X,y 0.2 ；L2；10 fold across MFCC

multi_logreg = LogisticRegression(multi_class="multinomial", solver="lbfgs",max_iter=1000)

param_grid = {
    'C': [ 1,10 ],
    'penalty': ['l2']
}

# 10 fold across
grid_search = GridSearchCV(multi_logreg, param_grid, cv=10, n_jobs=-1)

grid_search.fit(X_train_scaled, y_train)
print("Best parameters:", grid_search.best_params_)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)



target_names = [f'Class {i}' for i in range(36)]

# report
cr = classification_report(y_test, y_pred, target_names=target_names)

# Top-N accuracy score
N_values = [1, 2, 3, 4, 5]  # You can add more N values if needed
top_n_accuracies = [top_k_accuracy_score(y_test, best_model.predict_proba(X_test_scaled), k=N) for N in N_values]

# Print top-N accuracy scores
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")

# Print classification report
print("Classification Report:")
print(cr)

Best parameters: {'C': 10, 'penalty': 'l2'}
Top-1 Accuracy Score: 0.3707
Top-2 Accuracy Score: 0.4490
Top-3 Accuracy Score: 0.5136
Top-4 Accuracy Score: 0.5986
Top-5 Accuracy Score: 0.6701
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.40      0.25      0.31         8
     Class 1       0.46      0.67      0.55         9
     Class 2       0.27      0.44      0.33         9
     Class 3       0.40      0.40      0.40        10
     Class 4       0.57      0.44      0.50         9
     Class 5       0.33      0.12      0.18         8
     Class 6       0.25      0.22      0.24         9
     Class 7       0.41      0.70      0.52        10
     Class 8       0.53      0.80      0.64        10
     Class 9       0.33      0.30      0.32        10
    Class 10       0.40      0.29      0.33         7
    Class 11       0.23      0.33      0.27         9
    Class 12       0.22      0.25      0.24         8
    Class 13       0.75      0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### random forest

In [ ]:
import pydot
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from pprint import pprint
from sklearn import metrics
from openpyxl import load_workbook
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import classification_report

# Assuming you have loaded X_train_scaled, y_train, X_test_scaled, and y_test

# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [ 100],
    'max_depth': [None],
    'min_samples_split': [10],
    'min_samples_leaf': [  4],
    'bootstrap': [False]
}



# Create and fit the Random Forest model using GridSearchCV
random_forest = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(random_forest, param_grid, cv=10, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

best_random_forest = grid_search.best_estimator_

# Perform 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
y_test_pred_probs = cross_val_predict(best_random_forest, X_test_scaled, y_test, cv=kf, method='predict_proba')

# ... (Rest of your code for calculating top-N accuracy scores and generating the classification report)
N_values = [1, 2, 3, 4, 5]
top_n_accuracies = []
for N in N_values:
    top_n_correct = 0
    for true_label, pred_probs in zip(y_test, y_test_pred_probs):
        top_n_indices = np.argsort(pred_probs)[::-1][:N]  # Indices of top-N classes
        if true_label in top_n_indices:
            top_n_correct += 1
    top_n_accuracy = top_n_correct / len(y_test)
    top_n_accuracies.append(top_n_accuracy)

# Print the best hyperparameters found by GridSearchCV
print("Best Hyperparameters:")
print(grid_search.best_params_)

# Print top-N accuracy scores
for N, accuracy in zip(N_values, top_n_accuracies):
    print(f"Top-{N} Accuracy Score: {accuracy:.4f}")

# Generate classification report
target_names = [f'Class {i}' for i in range(36)]
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
cr = classification_report(y_test, y_test_pred, target_names=target_names)
# Print classification report
print("Classification Report:")
print(cr)



Best Hyperparameters:
{'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Top-1 Accuracy Score: 0.0612
Top-2 Accuracy Score: 0.1020
Top-3 Accuracy Score: 0.1361
Top-4 Accuracy Score: 0.1429
Top-5 Accuracy Score: 0.1735
Classification Report:
              precision    recall  f1-score   support

     Class 0       0.00      0.00      0.00         8
     Class 1       0.00      0.00      0.00         9
     Class 2       0.00      0.00      0.00         9
     Class 3       0.00      0.00      0.00        10
     Class 4       0.00      0.00      0.00         9
     Class 5       0.00      0.00      0.00         8
     Class 6       0.00      0.00      0.00         9
     Class 7       0.06      0.10      0.07        10
     Class 8       0.00      0.00      0.00        10
     Class 9       0.20      0.20      0.20        10
    Class 10       0.00      0.00      0.00         7
    Class 11       0.00      0.00      0.00         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## graph

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

# 创建一个包含您的数据的DataFrame
data = {
    'Metric': ['1', '2', '3', '4', '5'],
    'SVM': [0.8571, 0.9354, 0.949, 0.966, 0.966],
    'MLR': [0.8673, 0.932, 0.9626, 0.9728, 0.9762],
    'Random Forest': [0.2789, 0.3946, 0.4558, 0.5, 0.5544]
}

df = pd.DataFrame(data)

# 定义不同的标记形状
marker_shapes = ['o', 's', 'D']

# 绘制图表
plt.figure(figsize=(10, 6))

# 绘制每个模型的不同准确率得分
for col in df.columns[1:]:
    plt.plot(df['Metric'], df[col], marker='o', label=col)

plt.title('Accuracy Scores for Different Models')
plt.xlabel('number of guesses')
plt.ylabel('Accuracy Score')
plt.ylim(0, 1)  # 设置y轴范围
plt.legend()
plt.grid(True)
# plt.xticks(rotation=45)

# 添加百分比格式化器
plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1, decimals=1))

plt.tight_layout()

plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

# 创建一个包含您的数据的DataFrame
data = {
    'Metric': ['1', '2', '3', '4', '5'],
    'SVM': [0.8571, 0.9354, 0.949, 0.966, 0.966],
    'MLR': [0.8673, 0.932, 0.9626, 0.9728, 0.9762],
    'Random Forest': [0.2789, 0.3946, 0.4558, 0.5, 0.5544]
}

df = pd.DataFrame(data)

# 定义不同的标记形状
marker_shapes = ['o', 's', 'D']

# 绘制图表
plt.figure(figsize=(10, 6))

# 绘制每个模型的不同准确率得分
for i, col in enumerate(df.columns[1:], start=1):
    plt.plot(df['Metric'], df[col], marker=marker_shapes[i-1], label=col)

plt.title('Accuracy Scores for Different Models')
plt.xlabel('number of guesses')
plt.ylabel('Accuracy Score')
plt.ylim(0, 1)  # 设置y轴范围
plt.legend()
plt.grid(True)

# 添加百分比格式化器
plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1, decimals=1))

plt.tight_layout()

plt.show()


In [ ]:
data_shift = {
    'Metric': ['1', '2', '3', '4', '5'],
    'SVM': [0.8605, 0.9388, 0.9524,0.9626, 0.9660],
    'MLR': [0.8605, 0.9320,  0.9592, 0.9694, 0.9796],
    'Random Forest': [0.2789, 0.3946, 0.4558, 0.5, 0.5544]
    }
accuracy_datashift = {
    'SVM':[0.86],
    'MLR':[0.86],
    'Random Forest':[0.28]
}
